In [14]:
import sys
print(sys.version)

3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]


# Imports
All import files have been saved to import_file.py

In [2]:
from import_file import *

# Getting column descriptions
https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5/tables/1

In [10]:
col_names = ["Feature", "Explanation", "Measurement", "Range"]
df = pd.DataFrame(columns=col_names)

url = "https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5/tables/1"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')


table = soup.find("table", { "class" : "data last-table" })

with open("../CSV/column_details.html", "w") as file:
#     print(table.prettify())
    file.write(table.prettify())

df = pd.read_html(str(table))[0]

df.to_csv("../CSV/column_details.csv", index=False)
display( pd.read_csv("../CSV/column_details.csv") )

,Feature,Explanation,Measurement,Range
0,Age,Age of the patient,Years,"[40,..., 95]"
1,Anaemia,Decrease of red blood cells or hemoglobin,Boolean,"0, 1"
2,High blood pressure,If a patient has hypertension,Boolean,"0, 1"
3,Creatinine phosphokinase,Level of the CPK enzyme in the blood,mcg/L,"[23,..., 7861]"
4,(CPK),NaN,NaN,NaN
5,Diabetes,If the patient has diabetes,Boolean,"0, 1"
6,Ejection fraction,Percentage of blood leaving,Percentage,"[14,..., 80]"
7,NaN,the heart at each contraction,NaN,NaN
8,Sex,Woman or man,Binary,"0, 1"
9,Platelets,Platelets in the blood,kiloplatelets/mL,"[25.01,..., 850.00]"


# Cleaning Data
https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

In [11]:
data = pd.read_csv("../CSV/heart_failure_clinical_records_dataset.csv")

# feature = "density"
target = "DEATH_EVENT"
# color_var = "quality"

# No NA cells
# for c in data.columns:
#     print(c, data[c].isna().sum())

# All int64 and float64 columns
# print(data.dtypes)

# cols = list(data.columns)
# cols.remove("color")
# Handling outliers (Droopped 20 rows)
last_size = data.shape
print(data.shape)
for col in data.columns:
    col_std = data[col].std()
    col_mean = data[col].mean()

    col_min = col_mean - 3*col_std
    col_max = col_mean + 3*col_std

    data.drop(data[data[col] < col_min].index, inplace=True)
    data.drop(data[data[col] > col_max].index, inplace=True)
    
    if last_size != data.shape:
        print("Dropped", last_size[0] - data.shape[0] ,"row(s) from", col)
        last_size = data.shape
print(data.shape)

# Not sure why age is in Year but casted as float
data["age"] = data["age"].astype(int)
    
data.to_csv("../CSV/heart_failure_clinical_records_dataset_cleaned.csv", index=False)
display( pd.read_csv("../CSV/heart_failure_clinical_records_dataset_cleaned.csv") )
display( pd.read_csv("../CSV/heart_failure_clinical_records_dataset_cleaned.csv").dtypes )

(299, 13)
Dropped 7 row(s) from creatinine_phosphokinase
Dropped 1 row(s) from ejection_fraction
Dropped 3 row(s) from platelets
Dropped 6 row(s) from serum_creatinine
Dropped 3 row(s) from serum_sodium
(279, 13)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75,0,582,0,20,1,265000.0,1.9,130,1,0,4,1
1,65,0,146,0,20,0,162000.0,1.3,129,1,1,7,1
2,50,1,111,0,20,0,210000.0,1.9,137,1,0,7,1
3,90,1,47,0,40,1,204000.0,2.1,132,1,1,8,1
4,75,1,246,0,15,0,127000.0,1.2,137,1,0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,63,1,103,1,35,0,179000.0,0.9,136,1,1,270,0
275,62,0,61,1,38,1,155000.0,1.1,143,1,1,270,0
276,55,0,1820,0,38,0,270000.0,1.2,139,0,0,271,0
277,45,0,2413,0,38,0,140000.0,1.4,140,1,1,280,0


age                           int64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object

# Logistic Regression Model
The linear regression model had an $R^2 = 0.39$, where the logistic regression scored accuracy $0.807$. Since the variable we are predicting is binary the Logistic regression model makes sense as the more appropriate choice.

In [5]:
X = pd.DataFrame(data.drop(columns=[target]))
Y = pd.DataFrame(data[target])

# display(X, Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=2020)
model_linear = LinearRegression()
model_linear = model_linear.fit(X_train, Y_train.values.ravel()) 
print("Linear Score:\t", model_linear.score(X_train, Y_train.values.ravel()))

model_logistic = LogisticRegression()
model_logistic = model_logistic.fit(X_train, Y_train.values.ravel()) 
print("Logistic Score:\t", model_logistic.score(X_train, Y_train.values.ravel()))

#probs = model_linear.predict(X_test)

Linear Score:	 0.39098595306808914
Logistic Score:	 0.8071748878923767


In [12]:
# data.hist(figsize=(16,20), bins=50, xlabelsize=8, ylabelsize=8);

In [13]:
# data_corr = data.corr()[target]
# print(data_corr)